In [1]:
import os
import time
import json

import concurrent
import numpy as np
import pandas as pd
from tqdm import tqdm
from evaluation_functions import evaluate
from openai_functions import extract_ade_terms
from sentence_transformers import SentenceTransformer

## Set Up

### Variables

In [2]:
extract_dir = "results/extract/"

In [4]:
# train
drug_file = 'data/TAC2017/train_drug_label_text.csv'
manual_file = 'data/TAC2017/train_drug_label_text_manual_ades.csv'

# test
# drug_file = 'data/TAC2017/test_drug_label_text.csv'
# manual_file = 'data/TAC2017/test_drug_label_text_manual_ades.csv'

In [5]:
drugs = pd.read_csv(drug_file)
manual_ades = pd.read_csv(manual_file)
set_type = drug_file.split('/')[2].split('_')[0] # assuming file follows format "train_..." or "test...."

all_sections = drugs.query("section_name != 'all-concat'").groupby('drug_name')['section_text'].apply(' '.join).reset_index()
all_sections.insert(1, "section_name", ["all-concat" for _ in range(all_sections.shape[0])])
drugs = pd.concat([drugs, all_sections])

set_type

'train'

In [6]:
drugs.shape

(340, 3)

## Run GPT

In [7]:
outputs = {}

In [8]:
config = json.load(open('./config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint'] 

gpt_model = config[api_source]["gpt_model"]
# gpt_model = "gpt-4-turbo-preview"
# gpt_model = "gpt-3.5-turbo-0125"

temperature = 0

- fatal-one-shot_pharmexpert-v1_temp0_train_run0
- one-shot_gpt-written-prompt_temp0_train_run0
- two-shot_gpt-written-prompt_temp0_train_run0

In [35]:
nruns = 3

system_options = {
    "pharmexpert-v1": "You are an expert in medical natural language processing, adverse drug reactions, pharmacology, and clinical trials.",
    "gpt-written-prompt": """
Given the structured product label below, extract information on adverse drug reactions and provide the exact mentions of reactions
in a comma-separated format. Consider sentence-form expressions, reactions in tables, negated reactions, discontinuous mentions,
hypothetical scenarios, and potentially fatal occurrences. Ensure the extraction is comprehensive, covering both explicit and 
implicit references to adverse reactions. Each input will contain text from a single structured product label.
"""
    
}

prompt_options = {
    "one-shot":"""
Input: 'Information:\n\n\n\n *  Peripheral Neuropathy [see  Warnings and Precautions (      5.1      )  ]  \n *  Anaphylaxis and Infusion Reactions [see  Warnings and Precautions (      5.2      )  ]  \n *  Hematologic Toxicities [see  Warnings and Precautions (      5.3      )  ]  \n *  Serious Infections and Opportunistic Infections [see  Warnings and Precautions (      5.4      )  ]  \n *  Tumor Lysis Syndrome [see  Warnings and Precautions (      5.5      )  ]  \n *  Increased Toxicity in the Presence of Severe Renal Impairment [see  Warnings and Precautions (      5.6      )  ]  \n *  Increased Toxicity in the Presence of Moderate or Severe Hepatic Impairment [see  Warnings and Precautions (      5.7      )  ]  \n *  Hepatotoxicity [see  Warnings and Precautions]' 
Answer: 'peripheral neuropathy, anaphylaxis, infusion reactions, hematologic toxicities, infections, opportunistic infections, tumor lysis syndrome, increased toxicity, increased toxicity, hepatotoxicity'
Input: '{}'
""",
    "two-shot":"""
Input: 'Information:\n\n\n\n *  Peripheral Neuropathy [see  Warnings and Precautions (      5.1      )  ]  \n *  Anaphylaxis and Infusion Reactions [see  Warnings and Precautions (      5.2      )  ]  \n *  Hematologic Toxicities [see  Warnings and Precautions (      5.3      )  ]  \n *  Serious Infections and Opportunistic Infections [see  Warnings and Precautions (      5.4      )  ]  \n *  Tumor Lysis Syndrome [see  Warnings and Precautions (      5.5      )  ]  \n *  Increased Toxicity in the Presence of Severe Renal Impairment [see  Warnings and Precautions (      5.6      )  ]  \n *  Increased Toxicity in the Presence of Moderate or Severe Hepatic Impairment [see  Warnings and Precautions (      5.7      )  ]  \n *  Hepatotoxicity [see  Warnings and Precautions]' 
Answer: 'peripheral neuropathy, anaphylaxis, infusion reactions, hematologic toxicities, infections, opportunistic infections, tumor lysis syndrome, increased toxicity, increased toxicity, hepatotoxicity'
Input: 'Peripheral neuropathy : Monitor patients for neuropathy and institute dose modifications accordingly  (5.1)  . \n *   Anaphylaxis and infusion reactions : If an infusion reaction occurs, interrupt the infusion.  If anaphylaxis occurs, immediately discontinue the infusion  (5.2)  . \n *   Hematologic toxicities : Monitor complete blood counts prior to each dose of ADCETRIS.  Closely monitor patients for fever.  If Grade 3 or 4 neutropenia develops, consider dose delays, reductions, discontinuation, or G-CSF prophylaxis with subsequent doses  (5.3)  . \n *   Serious infections and oppo'
Answer: 'peripheral neuropathy, anaphylaxis, infusion reactions, hematologic toxicities, infections'
Input: {}
""",
    "fatal-one-shot":"""
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list.
If a fatal event is listed add 'death' to the list.
Input: 'Information:\n\n\n\n *  Peripheral Neuropathy [see  Warnings and Precautions (      5.1      )  ]  \n *  Anaphylaxis and Infusion Reactions [see  Warnings and Precautions (      5.2      )  ]  \n *  Hematologic Toxicities [see  Warnings and Precautions (      5.3      )  ]  \n *  Serious Infections and Opportunistic Infections [see  Warnings and Precautions (      5.4      )  ]  \n *  Tumor Lysis Syndrome [see  Warnings and Precautions (      5.5      )  ]  \n *  Increased Toxicity in the Presence of Severe Renal Impairment [see  Warnings and Precautions (      5.6      )  ]  \n *  Increased Toxicity in the Presence of Moderate or Severe Hepatic Impairment [see  Warnings and Precautions (      5.7      )  ]  \n *  Hepatotoxicity [see  Warnings and Precautions]' 
Answer: 'peripheral neuropathy, anaphylaxis, infusion reactions, hematologic toxicities, infections, opportunistic infections, tumor lysis syndrome, increased toxicity, increased toxicity, hepatotoxicity'
Input: '{}'
""",
"fatal-two-shot":"""
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list.
If a fatal event is listed add 'death' to the list.
Question: 'Information:\n\n\n\n *  Peripheral Neuropathy [see  Warnings and Precautions (      5.1      )  ]  \n *  Anaphylaxis and Infusion Reactions [see  Warnings and Precautions (      5.2      )  ]  \n *  Hematologic Toxicities [see  Warnings and Precautions (      5.3      )  ]  \n *  Serious Infections and Opportunistic Infections [see  Warnings and Precautions (      5.4      )  ]  \n *  Tumor Lysis Syndrome [see  Warnings and Precautions (      5.5      )  ]  \n *  Increased Toxicity in the Presence of Severe Renal Impairment [see  Warnings and Precautions (      5.6      )  ]  \n *  Increased Toxicity in the Presence of Moderate or Severe Hepatic Impairment [see  Warnings and Precautions (      5.7      )  ]  \n *  Hepatotoxicity [see  Warnings and Precautions]' 
Answer: 'peripheral neuropathy, anaphylaxis, infusion reactions, hematologic toxicities, infections, opportunistic infections, tumor lysis syndrome, increased toxicity, increased toxicity, hepatotoxicity'

Question: 'Peripheral neuropathy : Monitor patients for neuropathy and institute dose modifications accordingly  (5.1)  . \n *   Anaphylaxis and infusion reactions : If an infusion reaction occurs, interrupt the infusion.  If anaphylaxis occurs, immediately discontinue the infusion  (5.2)  . \n *   Hematologic toxicities : Monitor complete blood counts prior to each dose of ADCETRIS.  Closely monitor patients for fever.  If Grade 3 or 4 neutropenia develops, consider dose delays, reductions, discontinuation, or G-CSF prophylaxis with subsequent doses  (5.3)  . \n *   Serious infections and oppo'
Answer: 'peripheral neuropathy, anaphylaxis, infusion reactions, hematologic toxicities, infections'

Question: '{}'
Answer:
"""
}

system_name = "gpt-written-prompt"
system_content = system_options[system_name]

prompt_name = "two-shot"
prompt = prompt_options[prompt_name]

gpt_params = [f"temp{temperature}"]

output_file_basename = '{}_{}_{}_{}_{}_{}'.format(api_source, gpt_model, prompt_name, system_name, '-'.join(gpt_params), set_type)
output_file_basename

'OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train'

In [36]:
# run GPT
for i in range(nruns):
    run_key = "{}_run{}".format(output_file_basename, i)
    print(run_key)
    if run_key in outputs:
        print(f"Run {run_key} already started will pick up from where it was left off.")
    elif os.path.exists('{}/{}.csv'.format(extract_dir, run_key)):
        gpt_output = pd.read_csv('{}/{}.csv'.format(extract_dir, run_key))
        outputs[run_key] = gpt_output
        print(f"Run {run_key} started, loading from disk and pick up from where it was left off.")
    
    start = time.time()
    results = list()
    rows_to_run = list()
    for _, row in drugs.iterrows():
        name, section = row['drug_name'], row['section_name']

        if run_key in outputs:
            prev_run_results = outputs[run_key].query(f"drug_name == '{name}'").query(f"section_name == '{section}'")
            if prev_run_results.shape[0]==1:
                results.append([name, section, prev_run_results.gpt_output.values[0]])
            else:
                rows_to_run.append(row)
        else:
            rows_to_run.append(row)
        
    print(f"Loaded {len(results)} rows from file since they were already run.")
    print(f"There remains {len(rows_to_run)} rows to run.")

    def run_iteration(row):
        name, section = row['drug_name'], row['section_name']
        text = row['section_text']
        try:
            gpt_out = extract_ade_terms(api_source, config, gpt_model, system_content, prompt, text, temperature)
            return [name, section, gpt_out]
        except Exception as err:
            print(f"Encountered an exception for row: {name} {section}. Error message below:")
            print(err)
            return None
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as exec:
        results.extend(list(tqdm(
            exec.map(run_iteration, rows_to_run), 
            total=len(rows_to_run)
        )))
            
    gpt_output = pd.DataFrame(
        [r for r in results if r is not None],
        columns=['drug_name', 'section_name', 'gpt_output']
    )
    end = time.time()
    
    if gpt_output.shape[0] > 0:
        outputs[run_key] = gpt_output
        gpt_output.to_csv('{}/{}.csv'.format(extract_dir, run_key))
    
    print(f"Run: {run_key}, time elapsed: {end-start}s.")

OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run0
Run OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run0 already started will pick up from where it was left off.
Loaded 340 rows from file since they were already run.
There remains 0 rows to run.


0it [00:00, ?it/s]


Run: OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run0, time elapsed: 0.6736369132995605s.
OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run1
Loaded 0 rows from file since they were already run.
There remains 340 rows to run.


100%|██████████| 340/340 [05:58<00:00,  1.06s/it]


Run: OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run1, time elapsed: 358.79613399505615s.
OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run2
Loaded 0 rows from file since they were already run.
There remains 340 rows to run.


100%|██████████| 340/340 [05:03<00:00,  1.12it/s]

Run: OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run2, time elapsed: 303.427766084671s.


## Evaluation

In [39]:
for run_key in sorted(outputs.keys()):
    print(run_key)

OpenAI_gpt-4-1106-preview_fatal-one-shot_pharmexpert-v1_temp0_train_run0
OpenAI_gpt-4-1106-preview_fatal-one-shot_pharmexpert-v1_temp0_train_run1
OpenAI_gpt-4-1106-preview_fatal-one-shot_pharmexpert-v1_temp0_train_run2
OpenAI_gpt-4-1106-preview_fatal-two-shot_pharmexpert-v1_temp0_train_run0
OpenAI_gpt-4-1106-preview_fatal-two-shot_pharmexpert-v1_temp0_train_run1
OpenAI_gpt-4-1106-preview_fatal-two-shot_pharmexpert-v1_temp0_train_run2
OpenAI_gpt-4-1106-preview_one-shot_gpt-written-prompt_temp0_train_run0
OpenAI_gpt-4-1106-preview_one-shot_gpt-written-prompt_temp0_train_run1
OpenAI_gpt-4-1106-preview_one-shot_gpt-written-prompt_temp0_train_run2
OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run0
OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run1
OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run2


In [38]:
evaluate(outputs, manual_ades, 'strict')
evaluate(outputs, manual_ades, 'lenient')

Running strict evaluation and saving results to disk.
OpenAI_gpt-4-1106-preview_one-shot_gpt-written-prompt_temp0_train_run0
saving results to results/evals/OpenAI_gpt-4-1106-preview_one-shot_gpt-written-prompt_temp0_train_run0_strict_granular.csv and results/evals/OpenAI_gpt-4-1106-preview_one-shot_gpt-written-prompt_temp0_train_run0_strict_overall.csv


100%|██████████| 101/101 [00:05<00:00, 20.09it/s]


OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run0
saving results to results/evals/OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run0_strict_granular.csv and results/evals/OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run0_strict_overall.csv


100%|██████████| 101/101 [00:05<00:00, 20.00it/s]


OpenAI_gpt-4-1106-preview_fatal-one-shot_pharmexpert-v1_temp0_train_run0
saving results to results/evals/OpenAI_gpt-4-1106-preview_fatal-one-shot_pharmexpert-v1_temp0_train_run0_strict_granular.csv and results/evals/OpenAI_gpt-4-1106-preview_fatal-one-shot_pharmexpert-v1_temp0_train_run0_strict_overall.csv


...

100%|██████████| 101/101 [00:39<00:00,  2.57it/s]


OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run1
saving results to results/evals/OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run1_lenient_granular.csv and results/evals/OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run1_lenient_overall.csv


100%|██████████| 101/101 [00:29<00:00,  3.41it/s]


OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run2
saving results to results/evals/OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run2_lenient_granular.csv and results/evals/OpenAI_gpt-4-1106-preview_two-shot_gpt-written-prompt_temp0_train_run2_lenient_overall.csv


100%|██████████| 101/101 [00:30<00:00,  3.29it/s]


In [12]:
# if using embeddings -- run this once:
# get embeddings for manual annotation --- this part is slow -- but should take <5 min
embed_model_name = 'llmrails/ember-v1'
embed_model = SentenceTransformer(embed_model_name)
man_embeds = embed_model.encode(manual_ades['reaction_string'].tolist())
manual_ades['embeds'] = list(man_embeds)

/home/uog2000/miniconda3/envs/llm_cpus/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/uog2000/miniconda3/envs/llm_cpus/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/uog2000/miniconda3/envs/llm_cpus/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [13]:
evaluate(outputs, manual_ades, 'embed', embed_model=embed_model, embed_model_name=embed_model_name)

Running embed evaluation and saving results to disk.
OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0_ember-v1_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0_ember-v1_overall.csv


100%|█████████████████████████████████████████| 101/101 [05:32<00:00,  3.29s/it]


OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1_ember-v1_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1_ember-v1_overall.csv


100%|█████████████████████████████████████████| 101/101 [05:20<00:00,  3.17s/it]


OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2_ember-v1_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2_ember-v1_overall.csv


100%|█████████████████████████████████████████| 101/101 [06:39<00:00,  3.96s/it]
